## Tests

This notebook runs some tests to verify that our code is working as intended. It was used a lot during development to iron out bugs. It is important to note that testing in our case is not trivial. The best we can do is to check that the code works properly for games with $T=2$ and $T=3$ stages and trust in our ability to write code that generalizes to any number of stages. However, basic functionality like constructing and extending the blockchain is easily verified.

In [1]:
#IMPORTS
#libraries
import numpy as np
import time
import import_ipynb

#notebooks
import blockchain as bc
import helper_functions as helfun
import payoff_matrix as pm

importing Jupyter notebook from blockchain.ipynb
importing Jupyter notebook from helper_functions.ipynb
importing Jupyter notebook from payoff_matrix.ipynb


### Draw a Random Blockchain 
Print the miners, blocks, the blockchain, and the longest chain.

In [6]:
T = 7 #time horizon
n = 2 #number of players

#SET A SEED
seed = int(np.random.randint(0, 1000, 1)[0])
print("seed:", seed, "\n")
np.random.seed(seed)

#GENERATE A RANDOM VALID BLOCKCHAIN
parents = helfun.pickParents(T, n)
winners = helfun.pickWinners(T, n)

B = bc.Blockchain(n, parents, winners) #build

#PRINT THE MINERS
B.printMiner(0)
B.printMiner(1)
print("")

#PRINT ALL BLOCKS
for t in range(1, T+1): #for each element in the blockchain
    B.sequence[t].printBlock()

#DRAW THE BLOCKCHAIN USING TREELIB
helfun.drawChain(B)
print("longest chains", B.longestchains)

seed: 994 

miner 0 wins stages [2 3 4]
miner 1 wins stages [1 5 6 7]

block 1 mined by miner 1, parent is block 0
block 2 mined by miner 0, parent is block 0
block 3 mined by miner 0, parent is block 1
block 4 mined by miner 0, parent is block 3
block 5 mined by miner 1, parent is block 1
block 6 mined by miner 1, parent is block 5
block 7 mined by miner 1, parent is block 5

b0
├── b1
│   ├── b3
│   │   └── b4
│   └── b5
│       ├── b6
│       └── b7
└── b2

longest chains [[0, 1, 3, 4], [0, 1, 5, 6], [0, 1, 5, 7]]


### Payoff Calculation

In [3]:
#PRINT THE NUMBER OF BLOCKS EACH MINER MINED
#we use the random blockchain generated above
for m in range(2):
    print(f"\nthe number of blocks mined by miner {m} in the chain ending in...")
    for t in range(1, T+1):
        print(f"... block {t} is {B.getPayoff(m, t)}")
print("")

#'TEST' THE FUNCTION 'finalExpectedPayoff()'
#calculates the expected payoff at the end of the game BEFORE nature chooses a the 'winning' chain
for m in range(2):
    print(f"final expected payoff for miner {m} is", pm.finalExpectedPayoff(B, m))


the number of blocks mined by miner 0 in the chain ending in...
... block 1 is 0
... block 2 is 1
... block 3 is 1
... block 4 is 2
... block 5 is 0
... block 6 is 0
... block 7 is 0

the number of blocks mined by miner 1 in the chain ending in...
... block 1 is 1
... block 2 is 0
... block 3 is 1
... block 4 is 1
... block 5 is 2
... block 6 is 3
... block 7 is 3

final expected payoff for miner 0 is 0.6666666666666666
final expected payoff for miner 1 is 2.3333333333333335


### Blockchain Extension
Take the random blockchain from above and append a block to it. Then print the new blockchain.

In [4]:
m = 0 #index of the miner who wins the next stage, arbitrarily chosen as '0'
t = T #index of the block the winning miner mines at, also arbitrarily chosen as 'T'

B_ext = bc.ExtendedBlockchain(B, m, t)

print("extended chain:")
B_ext.printMiner(0)
B_ext.printMiner(1)
helfun.drawChain(B_ext)

extended chain:
miner 0 wins stages [2 3 4 8]
miner 1 wins stages [1 5 6 7]

b0
├── b1
│   ├── b3
│   │   └── b4
│   └── b5
│       ├── b6
│       └── b7
│           └── b8
└── b2



### Payoff Matrix and Strategies
Print a payoff matrix and calculate the strategies for each player given that payoff matrix.

This section was used extensively to verify that the payoff calculation works. We calculated payoffs for all situations in the two- and three-stage games manually (on paper), and subsequently checked if the code is able to verify our results. This process obviously took a lot of time. 

The reader may experiment with different blockchains. Some examples are given below for blockchains with two, three, and four blocks. More examples can be found in `main.ipynb`. Any payoff matrix can be calculated with relative ease. Please be aware that the program will take a considerable amount of time when $T$ is much larger than the number of blocks in the specified blockchain (i.e. when looking many stages into the future).

In [5]:
#define variables
T = 4 #time horizon of the game
n = 2 #number of players, DO NOT CHANGE THIS!

#EXAMPLES
#BLOCKCHAIN WITH TWO BLOCKS
parents = np.array([0])
winners = np.array([0]) #miner 0 wins the first stage

#BLOCKCHAIN WITH THREE BLOCKS
#parents = np.array([0, 0]) #both blocks b1 and b2 are appended to the genesis block b0
#winners = np.array([0, 1]) #miner 0 wins the first stage, miner 1 the second

#BLOCKCHAIN WITH FOUR BLOCKS
#parents = np.array([0, 0, 2]) #blocks b1 and b2 are appended to the genesis block b0, b3 is appended to b2
#winners = np.array([0, 1, 1]) #miner 0 wins the first stage, miner 1 the second and third


### ASSERTIONS ###
assert winners.shape[0] == parents.shape[0], "arrays 'parents' and 'winners' must be of same length"
t = len(parents) + 1 #current stage
for s in range(t-1):
    assert parents[s] < s+1, "the parent of a block b_t must have a strictly lower index s < t"


print('T =', T)

B = bc.Blockchain(n, parents, winners)
helfun.drawChain(B)

B.printMiner(0)
B.printMiner(1)

start = time.process_time()
M = pm.intermediatePayoffMatrix(B, T, n)
end = time.process_time()

print(f"list of Nash equilibrium tuple(s) in stage {t}:", pm.getStrategies(B, T, n))
print("\npayoff matrix:\n\n", M)
print("\n", helfun.timeElapsed(start, end))



T = 4

b0
└── b1

miner 0 wins stages [1]
miner 1 wins stages []
list of Nash equilibrium tuple(s) in stage 2: [(1, 1)]

payoff matrix:

 [[[1.6875 1.1875]
  [2.     1.5   ]]

 [[2.1875 1.1875]
  [2.5    1.5   ]]]

 0.2 seconds
